# Определение возраста покупателей

Дан датасет APPA-REAL. Нужно построить модель, которая выдаст на нем результат MAE не выше 8.

## Исследовательский анализ данных

In [ ]:
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

Загрузим датасет


In [ ]:
labels = pd.read_csv('/datasets/faces/labels.csv')

In [ ]:
datagen = ImageDataGenerator(rescale=1./255)
gen_flow = datagen.flow_from_dataframe(
        dataframe=labels,
        directory='/datasets/faces/final_files/',
        x_col='file_name',
        y_col='real_age',
        target_size=(224, 224),
        batch_size=32,
        class_mode='raw',
        seed=12345) 

Посмотрим на распределение таргета


In [ ]:
labels['real_age'].describe()

In [ ]:
labels['real_age'].hist(bins=100)

Нормальное распределение со среднем в 30 лет

1) Всего 7591 фотография. Минимальный возраст 1 год, максимальный 100. Среднее 30 лет.
2) Вокруг этого среднего модель должна показывать наилучший результат. Качество должно падать при приближении к 10 лет, а потом снова расти из-за того что в выборки присутствует большое кол-во детских фотографий.
3) Так же качество должно падать при приближении к 100, за исключением всплесков, которые приходятся на круглые даты: 30, 40, 50, 60 лет. Следовательно модель должна показывать на таких значениях результат выше, чем на остальных. Что говорит о невысоком качестве выборки

Посмотрим примеры фотографий


In [ ]:
features, target = next(train_gen_flow)

In [ ]:
fig = plt.figure(figsize=(10,10))
for i in range(12):
    fig.add_subplot(4, 4, i+1)
    plt.imshow(features[i])
    plt.xticks([])
    plt.yticks([])
    plt.tight_layout()

Присутствуют фотографии с поворотом, с рамками, неполные фотографии.


## Обучение модели

Возьмем модель ResNet50. Скачаем веса от обучения на ImageNet. Будем обучать, как сверточный, так и полносвязный слой.
Код:

In [2]:
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.resnet import ResNet50
from keras.optimizers import Adam
import pandas as pd


datagen_train = ImageDataGenerator(
                        rescale=1./255,
                        validation_split=0.25,
                        horizontal_flip=True,
                        vertical_flip=True,
                        rotation_range=20
                        )

datagen_test = ImageDataGenerator(
                        rescale=1./255,
                        validation_split=0.25,
                        )

def load_train(path):
    labels = pd.read_csv(path + "labels.csv")

    train_datagen_flow = datagen_train.flow_from_dataframe(
        dataframe=labels,
        x_col='file_name',
        y_col='real_age',
        directory=path + "/final_files",
        target_size=(224, 224),
        batch_size=32,
        subset="training",
        class_mode='raw',
        seed=12345,
    )

    return train_datagen_flow    

def load_test(path):
    labels = pd.read_csv(path + "labels.csv")

    test_datagen_flow = datagen_test.flow_from_dataframe(
        dataframe=labels,
        x_col='file_name',
        y_col='real_age',
        directory=path + "/final_files",
        target_size=(224, 224),
        batch_size=32,
        subset="validation",
        class_mode='raw',
        seed=12345,
    )

    return test_datagen_flow    


def create_model(input_shape):
    
    backbone = ResNet50(input_shape=input_shape,
                        include_top=False,
                        weights='/datasets/keras_models/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5',
                        )                     

    model = Sequential()
    model.add(backbone)
    model.add(GlobalAveragePooling2D())
    # model.add(Dense(128, activator="relu"))
    model.add(Dense(1, activation='relu'))                     

    optimizer = Adam(learning_rate=0.0001)

    model.compile(
            optimizer=optimizer,
            loss = "mean_absolute_error",
            metrics=["mae"])

    return model

def train_model(model, train_data, test_data, batch_size=None, epochs=10,
               steps_per_epoch=None, validation_steps=None):

    model.fit(train_data, 
              validation_data=test_data,
              batch_size=batch_size, epochs=epochs,
              steps_per_epoch=steps_per_epoch,
              validation_steps=validation_steps,
              verbose=2, shuffle=True)

    return model         

Вывод модели:

<class 'keras.engine.sequential.Sequential'>

Epoch 1/15

2022-09-06 15:49:44.967157: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10

2022-09-06 15:49:45.296217: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7

    118s - loss: 12.0127 - mae: 12.0134 - val_loss: 24.6058 - val_mae: 25.5783

Epoch 2/15

    89s - loss: 8.5904 - mae: 8.5901 - val_loss: 24.1549 - val_mae: 24.7032

Epoch 3/15

    89s - loss: 7.7933 - mae: 7.7934 - val_loss: 11.5024 - val_mae: 12.7224

Epoch 4/15

    93s - loss: 7.3236 - mae: 7.3227 - val_loss: 4.9238 - val_mae: 9.0986

Epoch 5/15

    85s - loss: 6.9226 - mae: 6.9227 - val_loss: 7.5137 - val_mae: 7.5813

Epoch 6/15

    85s - loss: 6.6754 - mae: 6.6749 - val_loss: 14.1526 - val_mae: 7.6289

Epoch 7/15

    94s - loss: 6.4776 - mae: 6.4783 - val_loss: 8.0979 - val_mae: 7.3746

Epoch 8/15

    86s - loss: 6.1774 - mae: 6.1771 - val_loss: 8.4167 - val_mae: 7.3070

Epoch 9/15

    93s - loss: 6.0592 - mae: 6.0593 - val_loss: 7.1106 - val_mae: 6.4689

Epoch 10/15

    92s - loss: 5.8745 - mae: 5.8749 - val_loss: 6.2747 - val_mae: 7.2008

Epoch 11/15

    96s - loss: 5.6118 - mae: 5.6119 - val_loss: 6.9747 - val_mae: 8.1291

Epoch 12/15

    85s - loss: 5.6475 - mae: 5.6478 - val_loss: 9.3070 - val_mae: 7.4224

Epoch 13/15

    84s - loss: 5.3967 - mae: 5.3961 - val_loss: 6.6267 - val_mae: 6.4946

Epoch 14/15

    93s - loss: 5.3573 - mae: 5.3573 - val_loss: 4.4750 - val_mae: 7.9123

Epoch 15/15

    92s - loss: 5.1263 - mae: 5.1263 - val_loss: 5.2615 - val_mae: 6.2807

Test MAE: 6.2807


## Анализ обученной модели

Мы обучили модель ResNet50 на датасете APPA-REAL. Использовалась архитектура бэкбон + голова. Обучался, как сверточный, так и полносвязный слой.<br> Использовались предобученные веса от обучения на ImageNet.<br><br>Параметры обучения:<br> Размер батча = 32<br> Оптимизатор = Adam<br> Learning rate = 0.0001<br> Количество эпох = 15<br><br>В датасете фотографии людей соотнесены с их возрастом(таргет). Мы добавили следующие аугментации:<br> -горизонтальный флип <br> -вертикалальный флип <br> -поворот до 20 градусов <br><br> Была задача получить MAE меньше "8". При обучении сеть несколько раз переобучалась, но через эпоху показывала все более лучший результат. В нашем случае модель показала результат в "6.2". Это значит, что в среднем на каждой фотографии модель будет ошибаться в "6.2" лет.